In [ ]:
import functions as fc
import pandas as pd

In [ ]:
fc.download_tgz_raw()

In [ ]:
fc.import_tgz("2018", "./data2018")

In [ ]:
fc.import_tgz("2019", "./data2019")

In [ ]:
fc.import_tgz("2020", "./data2020")

In [ ]:
df_final = fc.transform_rcs("2018")

In [ ]:
df_final.head()

In [ ]:
fc.export_2_minio("2018", df_final)

In [ ]:
fc.export_2_minio("2019", df_final)

In [ ]:
fc.export_2_minio("2020", df_final)